In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [3]:
prompt="""
Scenario: SearchWithUnavailableKeywordNotMatchingAnyShopBrandOrProduct

Search feature

Objectives: This use case allows user to search for what product, brand or shop they are interested in.

Preconditions: User is in Home page

Main flow: 
Step 1: User types a product name in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows a list of shops that sells the product and list of all the related products from many shops.

Alternative flow 1:
Step 1: User types a brand name in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows list of all the products from that brand from many shops. If that brand has it own official stores, system will shows that store

Alternative flow 2:
Step 1: User types a shop name in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows all the shop that has the same name and product that has the same word that user typed in.

Exception flow 1:
Step 1: User types an unavailable keyword that is not the same with any shop, brand or product available in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows a blank list and recommend user to try another keywords.

Exception flow 2:
Step 1: User types an unavailable keyword that is not the same with any brand or product available in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows all the shop that has the same name a blank product list and recommend user to try another keywords.

Exception flow 3:
Step 1: User types an unavailable keyword that is not the same with any shop available in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows a blank shop list recommend user to try another keywords and shows list of all the related products from many shops.

"""

In [4]:
promptTestCase = [
    { "role": "system", "content": SYSTEM_PROMPT_8},
    { "role": "user", "content": prompt}
  ]
gpt_response = ask(promptTestCase, client, model)
print(gpt_response)

{
  "Test Case 1": {
    "testCaseName": "Search with an unavailable keyword not matching any shop, brand, or product",
    "objective": "Verify that the system shows a blank list and recommends the user to try another keyword when an unavailable keyword is entered.",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type an unavailable keyword that does not match any shop, brand, or product into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a blank list and recommend the user to try another keyword.",
    "explanation": "This test case ensures that when users search for a keyword that does not match any shop, brand, or product, the system correctly handles the query by showing a blank list and suggesting the user to try different keywords. It directly matches the scenario name by testing the search functionality with an unavailable keyword."
  },
  "Test Case 2": {
    "tes

In [8]:
prompt="""
Scenario: Searching by Brand Name Scenario

Search feature

Objectives: This use case allows user to search for what product, brand or shop they are interested in.

Preconditions: User is in Home page

Main flow: 
Step 1: User types a product name in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows a list of shops that sells the product and list of all the related products from many shops.

Alternative flow 1:
Step 1: User types a brand name in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows list of all the products from that brand from many shops. If that brand has it own official stores, system will shows that store

Alternative flow 2:
Step 1: User types a shop name in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows all the shop that has the same name and product that has the same word that user typed in.

Exception flow 1:
Step 1: User types an unavailable keyword that is not the same with any shop, brand or product available in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows a blank list and recommend user to try another keywords.

Exception flow 2:
Step 1: User types an unavailable keyword that is not the same with any brand or product available in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows all the shop that has the same name a blank product list and recommend user to try another keywords.

Exception flow 3:
Step 1: User types an unavailable keyword that is not the same with any shop available in the search bar.
Step 2: User clicks on the search icon or press enter.
Step 3: System shows a blank shop list recommend user to try another keywords and shows list of all the related products from many shops.

"""

In [9]:
promptTestCase = [
    { "role": "system", "content": SYSTEM_PROMPT_8},
    { "role": "user", "content": prompt}
  ]
gpt_response = ask(promptTestCase, client, model)
print(gpt_response)

{
  "Test Case 1": {
    "testCaseName": "Search by Brand Name and Display Products",
    "objective": "Verify that the system displays all products from the searched brand and its official store if available",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type a known brand name into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a list of all the products from the searched brand from many shops. If the brand has its own official store, the system should also display that store.",
    "explanation": "This test case directly matches the scenario name 'Searching by Brand Name' and follows the alternative flow 1, ensuring the search functionality works as expected for brand names."
  },
  "Test Case 2": {
    "testCaseName": "Search by Brand Name with No Official Store",
    "objective": "Verify that the system correctly displays products from a brand that does not have an 

In [5]:
SYSTEM_PROMPT="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}
"""

In [6]:
prompt = """
Scenario: SearchWithUnavailableKeywordNotMatchingAnyShopBrandOrProduct

{
  "Test Case 1": {
    "testCaseName": "Search with an unavailable keyword not matching any shop, brand, or product",
    "objective": "Verify that the system shows a blank list and recommends the user to try another keyword when an unavailable keyword is entered.",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type an unavailable keyword that does not match any shop, brand, or product into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a blank list and recommend the user to try another keyword.",
    "explanation": "This test case ensures that when users search for a keyword that does not match any shop, brand, or product, the system correctly handles the query by showing a blank list and suggesting the user to try different keywords. It directly matches the scenario name by testing the search functionality with an unavailable keyword."
  },
  "Test Case 2": {
    "testCaseName": "Search with an unavailable keyword not matching any brand or product but matches a shop name",
    "objective": "Verify that the system shows all shops matching the name with a blank product list and recommends the user to try another keyword.",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type an unavailable keyword that does not match any brand or product but matches a shop name into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display all shops that match the name with a blank product list and recommend the user to try another keyword.",
    "explanation": "This test case checks the system's response when the search keyword does not match any brand or product but matches a shop name. It verifies that the system can differentiate between matching shop names and non-matching product or brand names, and provides appropriate feedback to the user."
  },
  "Test Case 3": {
    "testCaseName": "Search with an unavailable keyword not matching any shop but matches a brand or product",
    "objective": "Verify that the system shows a blank shop list, recommends the user to try another keyword, and displays related products.",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type an unavailable keyword that does not match any shop but matches a brand or product into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a blank shop list, recommend the user to try another keyword, and show a list of all related products from many shops.",
    "explanation": "This test case is designed to verify the system's behavior when the search keyword matches a brand or product but not any shop. It ensures that the system can still provide relevant product suggestions while indicating that no shops match the search term."
  }
}
"""

In [7]:
promptTestCase = [
    { "role": "system", "content": SYSTEM_PROMPT},
    { "role": "user", "content": prompt}
  ]
gpt_response = ask(promptTestCase, client, model)
print(gpt_response)

{
  "Test Case 1": {
    "testCaseName": "Search with an unavailable keyword not matching any shop, brand, or product",
    "objective": "Verify that the system shows a blank list and recommends the user to try another keyword when an unavailable keyword is entered.",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type an unavailable keyword that does not match any shop, brand, or product into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a blank list and recommend the user to try another keyword.",
    "reflect": "yes",
    "explanation": "This test case directly reflects the given test scenario by testing the system's response to a search query that does not match any shop, brand, or product. The expected outcome of a blank list and a recommendation to try another keyword is exactly what the scenario aims to verify."
  },
  "Test Case 2": {
    "testCaseName": "Search w

In [12]:
prompt = """
Scenario: Searching by Brand Name Scenario
{
  "Test Case 1": {
    "testCaseName": "Search by Brand Name and Display Products",
    "objective": "Verify that the system displays all products from the searched brand and its official store if available",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type a known brand name into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a list of all the products from the searched brand from many shops. If the brand has its own official store, the system should also display that store.",
    "explanation": "This test case directly matches the scenario name 'Searching by Brand Name' and follows the alternative flow 1, ensuring the search functionality works as expected for brand names."
  },
  "Test Case 2": {
    "testCaseName": "Search by Brand Name with No Official Store",
    "objective": "Verify that the system correctly displays products from a brand that does not have an official store",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type a brand name known not to have an official store into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a list of all the products from the searched brand from many shops, without showing an official store.",
    "explanation": "This test case is created to ensure the system behaves correctly when a searched brand does not have an official store, which is a specific condition under the scenario 'Searching by Brand Name'."
  },
  "Test Case 3": {
    "testCaseName": "Search by Unavailable Brand Name",
    "objective": "Verify that the system shows appropriate feedback when searching for a brand name that does not exist",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type an unavailable brand name into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should show a blank list and recommend the user to try another keyword.",
    "explanation": "This test case follows the exception flow 1, testing the system's response to a search query for a brand name that does not match any available brands, products, or shops."
  }
}
"""

In [13]:
promptTestCase = [
    { "role": "system", "content": SYSTEM_PROMPT},
    { "role": "user", "content": prompt}
  ]
gpt_response = ask(promptTestCase, client, model)
print(gpt_response)

{
  "Test Case 1": {
    "testCaseName": "Search by Brand Name and Display Products",
    "objective": "Verify that the system displays all products from the searched brand and its official store if available",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type a known brand name into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a list of all the products from the searched brand from many shops. If the brand has its own official store, the system should also display that store.",
    "reflect": "yes",
    "explanation": "This test case directly addresses the scenario of searching by brand name, ensuring that the search functionality is properly tested for brand names, including the display of products and the official store if available."
  },
  "Test Case 2": {
    "testCaseName": "Search by Brand Name with No Official Store",
    "objective": "Verify that the system c

In [14]:
prompt = """
Scenario: Searching by Shop Name Instead of Product Name
{
  "Test Case 1": {
    "testCaseName": "Search by Brand Name and Display Products",
    "objective": "Verify that the system displays all products from the searched brand and its official store if available",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type a known brand name into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a list of all the products from the searched brand from many shops. If the brand has its own official store, the system should also display that store.",
    "explanation": "This test case directly matches the scenario name 'Searching by Brand Name' and follows the alternative flow 1, ensuring the search functionality works as expected for brand names."
  },
  "Test Case 2": {
    "testCaseName": "Search by Brand Name with No Official Store",
    "objective": "Verify that the system correctly displays products from a brand that does not have an official store",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type a brand name known not to have an official store into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a list of all the products from the searched brand from many shops, without showing an official store.",
    "explanation": "This test case is created to ensure the system behaves correctly when a searched brand does not have an official store, which is a specific condition under the scenario 'Searching by Brand Name'."
  },
  "Test Case 3": {
    "testCaseName": "Search by Unavailable Brand Name",
    "objective": "Verify that the system shows appropriate feedback when searching for a brand name that does not exist",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type an unavailable brand name into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should show a blank list and recommend the user to try another keyword.",
    "explanation": "This test case follows the exception flow 1, testing the system's response to a search query for a brand name that does not match any available brands, products, or shops."
  }
}
"""

In [15]:
promptTestCase = [
    { "role": "system", "content": SYSTEM_PROMPT},
    { "role": "user", "content": prompt}
  ]
gpt_response = ask(promptTestCase, client, model)
print(gpt_response)

{
  "Test Case 1": {
    "testCaseName": "Search by Brand Name and Display Products",
    "objective": "Verify that the system displays all products from the searched brand and its official store if available",
    "testSteps": [
      "Step 1: Navigate to the Home page.",
      "Step 2: Type a known brand name into the search bar.",
      "Step 3: Click on the search icon or press enter."
    ],
    "expectedResult": "The system should display a list of all the products from the searched brand from many shops. If the brand has its own official store, the system should also display that store.",
    "reflect": "yes",
    "explanation": "This test case directly addresses the scenario of searching by shop (brand) name instead of product name by verifying that products from a specific brand and its official store are displayed after a search."
  },
  "Test Case 2": {
    "testCaseName": "Search by Brand Name with No Official Store",
    "objective": "Verify that the system correctly displ